# Notebook 1 — Zero-shot / One-shot / Few-shot (Hands-on)

Bu notebook, aynı görevi **3 strateji** ile çalıştırıp sonuçları kıyaslatır.

**Hedef:** format / tutarlılık / genelleme farkını canlı görmek.

⏱️ Önerilen süre: **12–15 dk**

## Setup

Bu notebook'lar, LLM çağrısı için **LangChain** kullanır ve otomatik sağlayıcı seçer:

- `OPENAI_API_KEY` varsa OpenAI üzerinden gider (varsayılan model: `gpt-4o-mini`)
- `GOOGLE_API_KEY` varsa Gemini üzerinden gider (varsayılan model: `gemini-1.5-pro`)

> Dilerseniz model adını `OPENAI_MODEL` / `GEMINI_MODEL` ile değiştirebilirsiniz.

**Not:** Bu notebook dosyaları örnek amaçlıdır. Kendi kurumsal ortamınızda anahtar yönetimini güvenli şekilde yapın.

In [ ]:
# Lokal/Colab için (gerekliyse) kurulum:
%pip -q install -U langchain-core langchain-openai langchain-google-genai pydantic tiktoken python-dotenv pandas matplotlib

In [ ]:
import os, json, re
from typing import Dict, Any

def get_llm():
    # Sağlayıcı seçimi: OpenAI -> Gemini
    openai_key = os.getenv('OPENAI_API_KEY')
    google_key = os.getenv('GOOGLE_API_KEY')

    if openai_key:
        from langchain_openai import ChatOpenAI
        model = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')
        llm = ChatOpenAI(model=model, temperature=0.1)
        return llm, f'OpenAI ({model})'

    if google_key:
        from langchain_google_genai import ChatGoogleGenerativeAI
        model = os.getenv('GEMINI_MODEL', 'gemini-1.5-pro')
        llm = ChatGoogleGenerativeAI(model=model, temperature=0.1, api_key=google_key)
        return llm, f'Google Gemini ({model})'

    raise RuntimeError(
        'API key bulunamadı. Lütfen ortam değişkeni tanımlayın:\n'
        '- OpenAI için: OPENAI_API_KEY\n'
        '- Gemini için: GOOGLE_API_KEY\n'
        'Opsiyonel: OPENAI_MODEL / GEMINI_MODEL'
    )

llm, provider = get_llm()
print('✅ LLM hazır:', provider)

def llm_text(prompt: str) -> str:
    resp = llm.invoke(prompt)
    return getattr(resp, 'content', str(resp)).strip()

def strip_fences(s: str) -> str:
    s = s.strip()
    s = re.sub(r'^```json\s*', '', s, flags=re.IGNORECASE)
    s = re.sub(r'^```\s*', '', s)
    s = re.sub(r'\s*```$', '', s)
    return s.strip()

In [ ]:
# Mini veri seti (email/ticket) — demo için
EMAILS = [
    {'id': 'E1', 'text': 'Kargom hâlâ gelmedi. 7 gündür bekliyorum. Acil çözüm istiyorum!', 'notes': 'Gecikme + yüksek aciliyet'},
    {'id': 'E2', 'text': 'Ürün kırık geldi. Değişim yapabilir miyiz?', 'notes': 'Hasarlı ürün'},
    {'id': 'E3', 'text': 'İade sürecini nasıl başlatabilirim? Kutuyu attım ama ürün duruyor.', 'notes': 'İade + edge-case (kutusuz)'},
    {'id': 'E4', 'text': 'Kartımdan iki kez çekim yapılmış görünüyor. Lütfen hemen kontrol edin.', 'notes': 'Faturalama + yüksek aciliyet'},
    {'id': 'E5', 'text': 'Ürününüzün kullanım kılavuzunu paylaşır mısınız?', 'notes': 'Bilgi talebi (low)'},
]
len(EMAILS)

## 1) Görev: Email triage → JSON

Çıktıyı bir **output contract** ile kilitliyoruz:
- `category`: sınıf adı
- `urgency`: low | medium | high
- `reason`: **tek cümle** gerekçe

Amaç: üretimde parse edilebilir, stabil çıktı.

In [ ]:
SCHEMA = (
    'Return ONLY valid JSON with exactly these keys:\n'
    '{\n'
    '  "category": "string",\n'
    '  "urgency": "low|medium|high",\n'
    '  "reason": "string (max 1 sentence)"\n'
    '}\n'
    'No extra text, no markdown, JSON only.'
)

import json

## 2) Prompt fonksiyonları

### Zero-shot
Örnek yok. Instruction + contract + input.

### One-shot
Tek örnekle output kalıbını ankore eder.

### Few-shot
3 örnekle pattern genellemesi + edge-case güçlenir.

In [ ]:
def prompt_zero_shot(email_text: str) -> str:
    return (
        'You are a customer support triage assistant.\n'
        'Classify the email into a category and urgency.\n\n'
        + SCHEMA + '\n\n'
        + 'Email:\n' + email_text
    )

ONE_EXAMPLE = {
    'email': 'My order arrived broken. I want a replacement as soon as possible.',
    'answer': {
        'category': 'Damaged product',
        'urgency': 'medium',
        'reason': 'Customer reports a damaged item and requests a replacement.'
    }
}

def prompt_one_shot(email_text: str) -> str:
    return (
        'You are a customer support triage assistant.\n'
        'Use the example to follow the same output format and labeling style.\n\n'
        + SCHEMA + '\n\n'
        + 'Example:\nEmail:\n' + ONE_EXAMPLE['email'] + '\n'
        + 'Answer:\n' + json.dumps(ONE_EXAMPLE['answer'], ensure_ascii=False) + '\n\n'
        + 'Now classify this email:\n\nEmail:\n' + email_text
    )

FEW_EXAMPLES = [
    (
        'Where is my package? It was supposed to arrive 5 days ago.',
        {'category':'Delivery issue','urgency':'high','reason':'Customer reports a significantly delayed delivery.'}
    ),
    (
        'How can I return the product? I changed my mind.',
        {'category':'Return request','urgency':'low','reason':'Customer asks for return instructions without a critical issue.'}
    ),
    (
        'You charged me twice for the same order. Fix this immediately.',
        {'category':'Billing issue','urgency':'high','reason':'Customer reports a double charge and requests urgent resolution.'}
    ),
]

def prompt_few_shot(email_text: str) -> str:
    ex_block = ''
    for mail, ans in FEW_EXAMPLES:
        ex_block += 'Email:\n' + mail + '\nAnswer:\n' + json.dumps(ans, ensure_ascii=False) + '\n\n'
    return (
        'You are a customer support triage assistant.\n'
        'Follow the same pattern as the examples.\n\n'
        + SCHEMA + '\n\n'
        + 'Examples:\n' + ex_block
        + 'Now classify this email:\n\nEmail:\n' + email_text
    )

## 3) Run + Parse

Çıktıyı JSON parse ediyoruz. Eğer model formatı bozarsa hata yakalanacak.

In [ ]:
def run_and_parse(prompt: str) -> Dict[str, Any]:
    raw = llm_text(prompt)
    raw2 = strip_fences(raw)
    try:
        return json.loads(raw2)
    except json.JSONDecodeError:
        print('⚠️ JSON parse edilemedi. Ham çıktı:\n', raw)
        raise

def run_strategy(name: str, make_prompt, email_text: str) -> Dict[str, Any]:
    print('\n===', name, '===')
    out = run_and_parse(make_prompt(email_text))
    print(json.dumps(out, ensure_ascii=False, indent=2))
    return out

## 4) Tek email üzerinde karşılaştırma

In [ ]:
test_email = EMAILS[0]['text']
print('TEST:', test_email)
_ = run_strategy('ZERO-SHOT', prompt_zero_shot, test_email)
_ = run_strategy('ONE-SHOT', prompt_one_shot, test_email)
_ = run_strategy('FEW-SHOT', prompt_few_shot, test_email)

## 5) Batch karşılaştırma (5 email)

Sonuçları tablo gibi görüp tutarlılığı tartışın.

In [ ]:
import pandas as pd

def batch_run(make_prompt):
    rows = []
    for e in EMAILS:
        out = run_and_parse(make_prompt(e['text']))
        rows.append({
            'id': e['id'],
            'notes': e['notes'],
            'category': out.get('category'),
            'urgency': out.get('urgency'),
            'reason': out.get('reason'),
        })
    return rows

df_zero = pd.DataFrame(batch_run(prompt_zero_shot)).add_prefix('zero_')
df_one  = pd.DataFrame(batch_run(prompt_one_shot)).add_prefix('one_')
df_few  = pd.DataFrame(batch_run(prompt_few_shot)).add_prefix('few_')

df = pd.concat([
    df_zero[['zero_id','zero_notes','zero_category','zero_urgency']],
    df_one[['one_category','one_urgency']],
    df_few[['few_category','few_urgency']],
], axis=1)
df

## 6) Egzersiz (3–5 dk)

1) `FEW_EXAMPLES` içine **edge-case** bir örnek ekleyin.
2) `category` adlarını sabitleyin (Delivery issue, Billing issue, ...).
3) Aynı email'i 3 kez çalıştırıp tutarlılığı inceleyin.

➡️ Sonraki notebook: **JSON output** + validation + repair.